# Import librerias

In [ ]:
!pip install shap

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import seaborn as sns
import missingno as msno

# Preprocessing
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from scipy import stats
from sklearn.feature_selection import VarianceThreshold
# Modelizacion
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

# Evaluacion
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, make_scorer
from sklearn.model_selection import train_test_split, cross_val_score, KFold, cross_val_score
from sklearn import metrics
from xgboost import plot_tree
import shap
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings("ignore")

import pickle
plt.rcParams["figure.figsize"] = [10,6]
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import rcParams
import seaborn as sns
import missingno as msno

from sklearn.impute import SimpleImputer
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import VarianceThreshold

#XGBoost
import xgboost as xgb
from xgboost import plot_tree

#Shap value
import shap
from scipy.stats import pearsonr

import warnings
warnings.filterwarnings('ignore')

Dejo algunas funciones de ayuda...

In [ ]:
def column_report(df_x):
    a = pd.DataFrame(df_x.dtypes, columns={'dtype'})
    b = pd.DataFrame(df_x.isna().sum(), columns={'nulls'})
    b['%nulls'] = round(100*b['nulls']/ df_x.shape[0],2)
    c = df_x.describe(include='all').transpose()
    return a.join(b).join(c).sort_values(by=['dtype','nulls'])


def evaluate_model(model, X_train, y_train, X_test, y_test):

    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)

    rmse_train = np.sqrt(metrics.mean_squared_error(y_train, pred_train))
    rmse_test = np.sqrt(metrics.mean_squared_error(y_test, pred_test))

    print('RMSE train:',rmse_train)
    print('RMSE test:',rmse_test)

    return rmse_train, rmse_test

# Import data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Read traintest dataset
df = pd.read_csv('/content/drive/MyDrive/Notebooks proyectos/Proyecto RENFE/renfe_traintest.csv')

In [ ]:
df.head()

In [ ]:
df.shape

# Preprocessing

## Elimina columnas y filas irrelevantes

In [ ]:
#df.drop(columns='Unnamed: 0',inplace=True)
print(df.shape)

In [ ]:
print(df.duplicated().sum(),'duplicated rows were deleted')
df.drop_duplicates(inplace=True)
print(df.shape)

## Data types

In [ ]:
df.info()

In [ ]:
# Pasar fechas de tipo objeto a tipo datetime
for i in ('departure','arrival','insert_date'):
  df[i] = pd.to_datetime(df[i])

# Split X-y

In [ ]:
# Divide dataset en matriz de variables (X) y output (y)
target = 'price'
features = df.columns[df.columns!=target]

X = df[features]
y = df[target]

## EDA: Analisis exploratorio

(breve)

In [ ]:
sns.distplot(y)
plt.title('Target Distribution: '+target)
plt.show()

In [ ]:
y.describe().T

In [ ]:
X.describe(include='all').T

## Feature Engineering

In [ ]:
# Variables de fechas las paso a cuatro variables distintas: día del mes, día de la semana, mes, hora.
for i in ('departure','insert_date'):
  X['weekday_'+i] = X[i].dt.day_name()
  X['weekday_n_'+i] = X[i].dt.day_of_week # 0 -Lunes, 6-Domingo
  X['month_'+i] = X[i].dt.month
  X['hour_'+i] = X[i].dt.hour

In [ ]:
# Tiempo entre la consulta del precio y el viaje
timedelta = X.departure - X.insert_date
X['days_before_trip'] = timedelta.dt.days

In [ ]:
festivos = pd.read_csv('/content/drive/MyDrive/Notebooks proyectos/Proyecto RENFE/dias_festivos.csv')

In [ ]:
festivos = pd.to_datetime(festivos.values.flatten(), format='%d-%m-%Y')

In [ ]:
X['ind_weekend'] = X.weekday_n_departure.isin([5,6])

In [ ]:
X['ind_holidays'] = X.departure.dt.date.isin(festivos.date)

In [ ]:
X.ind_holidays = X.ind_holidays.astype('int8')
X.ind_weekend = X.ind_weekend.astype('int8')

In [ ]:
X.info()

In [ ]:
# Hipotesis: los precios de las lineas de tren ida y vuelta son parecidos
# Vamos a ponerle nombre a las lineas de tren
# Definimos que el nombre de la linea será CIUDAD CON MAS VIAJES DE ORIGEN - CIUDAD CON MENOS VIAJES DE ORIGEN

city_order = X.origin.value_counts().index.values

def train_line(origin, destination, city_list):
  if np.where(city_order == origin) < np.where(city_order == destination):
    return origin+'-'+destination
  else:
    return destination+'-'+origin

# Aplicamos la funcion linea a linea en el dataframe
X['train_line'] = X.apply(lambda x: train_line(x['origin'], x['destination'], city_order), axis=1)

In [ ]:
X.head(3)

In [ ]:
X.info()

In [ ]:
X.columns

In [ ]:
100*X.vehicle_class.value_counts() / X.shape[0]

## Gestionar Nulos

In [ ]:
100*X.isna().sum()/X.shape[0]

In [ ]:
#Funcion de ayuda para tomar decisiones en la gestion de nulos
'''
def column_report(df_x):
    a = pd.DataFrame(df_x.dtypes, columns={'dtype'})
    b = pd.DataFrame(df_x.isna().sum(), columns={'nulls'})
    b['%nulls'] = round(100*b['nulls']/ df_x.shape[0],2)
    c = df_x.describe(include='all').transpose()
    return a.join(b).join(c).sort_values(by=['dtype','nulls'])
'''

In [ ]:
# report = column_report(df)

In [ ]:
# Analisis de nulos
'''
pd.set_option('display.max_rows', 1000)
report[report['nulls']>0].sort_values(by='nulls',ascending=False)
'''

Hay varias columnas con altos % de nulos. Como tenemos tantas variables lo aconsejable sería directamente eliminar aquellas con muchos nulos (>50
%). Antes de eliminar ninguna variable vamos a comprobar que no estén altamente correlacionada con el target:

In [ ]:
corr = df.corr()

In [ ]:
corr_list = pd.DataFrame(corr[target].abs().sort_values(ascending=False))
corr_list.rename(columns={target:'correlation'},inplace=True)
corr_list[corr_list['correlation']>0.50]

In [ ]:
X.drop(columns='seats',inplace=True)

## One-hot-encoding

In [ ]:
categorical = X.select_dtypes(include='object').columns

In [ ]:
X[categorical].nunique()

In [ ]:
for i in categorical:
  print(i)
  print(X[i].value_counts())
  print()

In [ ]:
#Definimos como residual todo valor con una frecuencia por debajo del 5%
min_weight = 0.05

for i in categorical:
  count = X[i].value_counts() / X.shape[0]
  most_frequent = count[count>min_weight].index.to_list()
  X[i] = X[i].apply(lambda x: x if x in most_frequent else 'OTRO')

In [ ]:
# Procedemos al one hot encoding
X = pd.get_dummies(data=X,
                               prefix=None,
                               prefix_sep="_",
                               columns=categorical,
                               dtype="int8")

In [ ]:
X.head(3)

In [ ]:
X.shape

In [ ]:
# Eliminamos las columnas antiguas de fechas pues ya no nos sirven
X.drop(columns=["departure", "arrival", "insert_date"], inplace=True)

In [ ]:
X.info()

## Altamente correlacionadas

In [ ]:
# Divide dataset en matriz de variables (X) y output (y)
features = X.columns[X.columns!=target]

In [ ]:
def highly_correlated(X, y, threshold):
    col_corr = list() # Set of all the names of deleted columns
    colnames = list()
    rownames = list()
    corr_matrix = X.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colnames.append(corr_matrix.columns[i]) # getting the name of column
                rownames.append(corr_matrix.index[j])
                col_corr.append(corr_matrix.iloc[i, j])
    Z = pd.DataFrame({'F1':colnames,
                      'F2':rownames,
                      'corr_F1_F2':col_corr,
                      'corr_F1_target': [np.abs(np.corrcoef(X[i],y)[0,1]) for i in colnames],
                      'corr_F2_target': [np.abs(np.corrcoef(X[i],y)[0,1]) for i in rownames]
                      })
    Z['F_to_delete'] = rownames
    Z['F_to_delete'][Z['corr_F1_target'] < Z['corr_F2_target']] = Z['F1'][Z['corr_F1_target'] < Z['corr_F2_target']]

    return Z

In [ ]:
# Detectar y elimanamos parejas de variables altamente correlacionadas
highly_corr = highly_correlated(X, y, 0.95)
highly_corr

## Eliminar variables baja Varianza

In [ ]:
vt = VarianceThreshold(threshold = 0.01) #Eliminamos columnas donde el 99% de los valores son iguales
vt.fit(X)
cols_lowvar = X.columns[vt.get_support()==False]
X.drop(columns=cols_lowvar,inplace=True)
print(len(cols_lowvar),' low variance features were removed:\n', cols_lowvar)

## Split train-test

Test debe ser 30%

In [ ]:
X.shape

In [ ]:
X.info()

In [ ]:
# Divide dataset en matriz de variables (X) y output (y)
features = X.columns[X.columns!=target]

In [ ]:
# Divide X-y en Train (70%) y Test (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print("Train: {:,}".format(len(y_train)))
print("Test: {:,}".format(len(y_test)))

# Torneo de modelos

* Probar regresion Lineal y xgbRegressor al menos
* Utilizar en cada modelo la funcion evaluate_model(model, X_train, y_train, X_test, y_test)

Modelo Regresión Lineal

In [ ]:
X_train3 = X_train.copy()
X_test3 = X_test.copy()
y_train3 = y_train.copy()
y_test3 = y_test.copy()

In [ ]:
# Estandarización: fit_transform() en train, transform() en test
scaler = StandardScaler()
X_train3 = scaler.fit_transform(X_train3) # el fit es para ajustar los datos, calcula la media y varianza del X_train y lo aplica sobre él pero solo a datos entrenados no a los del test.
X_test3 = scaler.transform(X_test3)

In [ ]:
X_train3

In [ ]:
X_test3

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train3,y_train3)

In [ ]:
lin_reg.coef_

In [ ]:
# Fórmula de la regresión lineal sobre las variables estandarizadas
print("MODEL:")  # son variables estandarizadas
print(target+" = "+" ".join(i+' x '+str(round(j,1))+' + ' for i,j in zip(X.columns.values,lin_reg.coef_))+str(round(lin_reg.intercept_,1)))

In [ ]:
#Predicción sobre X test
y_pred = lin_reg.predict(X_test3)

In [ ]:
error = y_pred - y_test.values

In [ ]:
RMSE = np.sqrt(np.mean(error**2))
RMSE

Modelo XGBReg

In [ ]:
model_XGBReg = xgb.XGBRegressor(colsample_bytree=0.5, n_estimators=500, objective='reg:squarederror')

In [ ]:
eval_set = [(X_train, y_train), (X_test, y_test)]
model_XGBReg.fit(
    X_train,
    y_train,
    eval_metric = "rmse",
    eval_set = eval_set,
    verbose = False
)

In [ ]:
y_pred_train = model_XGBReg.predict(X_train)
rmse_train = np.sqrt(np.mean((y_pred_train - y_train) ** 2))

y_pred_test = model_XGBReg.predict(X_test)
rmse_test = np.sqrt(np.mean((y_pred_test - y_test) ** 2))

print("RMSE Train: %.3f" %rmse_train)
print("RMSE Test: %.3f" %rmse_test)

In [ ]:
# Nos quedamos con el modelo XGBReg que da mejores resultados: 8.1 frente a 13.1  A continuación hiperparametrizamos

# Hiperparametrizacion: GridSearch

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb_GS = xgb.XGBRegressor()  #ahora XGBRegresssor

parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower Son los parámetros que utilizará y dara el mejor modelo con un comando posterior
              'objective':['reg:squarederror'],
              'learning_rate': [0.03, 0.05], #so called `eta` value
              'max_depth': [3, 5, 7],
              'min_child_weight': [4],
              'subsample': [0.7],
              'colsample_bytree': [0.5,0.7],
              'n_estimators': [100,500,1000]}


In [ ]:
xgb_grid = GridSearchCV(xgb_GS,
                        parameters,
                        cv=2,
                        verbose=True)

xgb_grid.fit(X_train, y_train)

In [ ]:
# Best model (XGBoost hiperparametrizado)
model_XGBReg_hp = xgb_grid.best_estimator_
model_XGBReg_hp

In [ ]:
#Prediction and metrics
y_pred_train = model_XGBReg_hp.predict(X_train)
rmse_train = np.sqrt(np.mean((y_pred_train - y_train) ** 2))

y_pred_test = model_XGBReg_hp.predict(X_test)
rmse_test = np.sqrt(np.mean((y_pred_test - y_test) ** 2))

print("RMSE Train: %.3f" %rmse_train)
print("RMSE Test: %.3f" %rmse_test)

In [ ]:
# El error baja bastante

In [ ]:
#Error distribution
error = y_pred_test - y_test.values
plt.figure(figsize=(10, 6))
plt.title('Distribucion del error (price)', fontsize=15)
sns.histplot(error)
plt.show()

In [ ]:
#Compara Prediccion vs dato real
plt.figure(figsize=[10,6])
plt.scatter(y_test,y_pred_test)
plt.plot(y_test,y_test,'r')
plt.title('True vs Predicted')
plt.xlabel('True values')
plt.ylabel('Predicted values')
plt.legend(['Regresion Ideal (0% Error)','Nuestra Regresión'])
plt.show()

In [ ]:
# el error es una campana de gauss perfectamente normalizada lo cual da idea de que el modelo es correcto

# Interpretabilidad del modelo

In [ ]:
# Visualiza un árbol concreto de los [n_estimators] que utiliza el XGBoost
plt.rcParams["figure.figsize"] =[20, 10]
plot_tree(model_XGBReg, num_trees=50)
plt.show()

In [ ]:
# Feature importance: numero de veces que emplea la variable para determinar el output
rcParams['figure.figsize'] = 12,8
sns.set(context="talk")
xgb.plot_importance(model_XGBReg_hp, max_num_features=20, importance_type='weight')
plt.show()

In [ ]:
# Las variables más importantes que obtenemos son: la duración, el tiempo antes del viaje, mes, hora y día de la salida.

In [ ]:
#SHAP value: peso en el output final,  impacto promedio en el output
explainer = shap.TreeExplainer(model_XGBReg_hp)
shap_values = explainer.shap_values(X_test)
plt.title('SHAP values')
shap.summary_plot(shap_values, X_test, plot_type="bar", max_display=20)

In [ ]:
# Respecto al output donde se tiene más impacto de media es en el tren línea Madrid-Barcelona, la tarifa flexible, el tren AVE y la duración.

In [ ]:
shap.summary_plot(shap_values, X_test)

# Predict

## OPCION A) Generar listado de predicts

Se pide hacer predicciones de precio para los datos de validacion y calcular el RMSE en validacion
Hay que replicar pasos del preprocessing en dataset validacion:

Cambios de formato,
Feature Engineering,
One hot encoding,
Eliminacion de columnas,
Estandarizacion de X (aplicar el mismo .transform)

In [ ]:
X_pred = pd.read_csv('/content/drive/MyDrive/Notebooks proyectos/Proyecto RENFE/renfe_predict.csv')

In [ ]:
X_pred.head(3)

In [ ]:
X_pred.shape

In [ ]:
X.columns.to_list()

In [ ]:
X_ejemplo = X.sample(n=4,random_state=30)
X_ejemplo

In [ ]:
predictions = model_XGBReg.predict(X_ejemplo)

In [ ]:
df_output = X_ejemplo.copy(deep=True)
df_output['PREDICTION Price'] = predictions
df_output

In [ ]:
# El modelo predice bien pues ha sido bien entrenado. Ahora pondremos el dataset de predicción y haremos todo el proceso de nuevo

In [ ]:
X_pred.info()

In [ ]:
X_pred

In [ ]:
# Gestión_duplicados
print(X_pred.duplicated().sum(),'duplicated rows were deleted')
X_pred.drop_duplicates(inplace=True)
print(X_pred.shape)

In [ ]:
#Funcion de ayuda para tomar decisiones en la gestion de nulos
'''
def column_report(df_x):
    a = pd.DataFrame(df_x.dtypes, columns={'dtype'})
    b = pd.DataFrame(df_x.isna().sum(), columns={'nulls'})
    b['%nulls'] = round(100*b['nulls']/ df_x.shape[0],2)
    c = df_x.describe(include='all').transpose()
    return a.join(b).join(c).sort_values(by=['dtype','nulls'])

report = column_report(X_pred)
'''

In [ ]:
# Analisis de nulos
'''
pd.set_option('display.max_rows', 1000)
report[report['nulls']>0].sort_values(by='nulls',ascending=False)
'''

In [ ]:
X_pred.info()

In [ ]:
#procedemos a eliminar la columna seats por la gran cantidad de nulos que tiene
del(X_pred['seats'])

In [ ]:
# Pasar fechas de tipo objeto a tipo datetime
for i in ('departure','arrival','insert_date'):
  X_pred[i] = pd.to_datetime(X_pred[i])

In [ ]:
X_pred.info()

In [ ]:
# Variables de las fechas
for i in ('departure','insert_date'):
  X_pred['weekday_'+i] = X_pred[i].dt.day_name()
  X_pred['weekday_n_'+i] = X_pred[i].dt.day_of_week # 0-Lunes, 6-Domingo
  X_pred['month_'+i] = X_pred[i].dt.month
  X_pred['hour_'+i] = X_pred[i].dt.hour

In [ ]:
# Tiempo entre la consulta del precio y el viaje
timedelta = X_pred.departure - X_pred.insert_date
X_pred['days_before_trip'] = timedelta.dt.days

In [ ]:
X_pred['ind_weekend'] = X_pred.weekday_n_departure.isin([5,6])

In [ ]:
X_pred['ind_holidays'] = X_pred.departure.dt.date.isin(festivos.date)

In [ ]:
X_pred.ind_holidays = X_pred.ind_holidays.astype('int8')
X_pred.ind_weekend = X_pred.ind_weekend.astype('int8')

In [ ]:
del(X_pred['Unnamed: 0'])

In [ ]:
X_pred.columns

In [ ]:
X_pred.origin.value_counts().index.values #ordena la frequencia de origen

In [ ]:
# Hipotesis: los precios de las lineas de tren ida y vuelta son parecidos
# Vamos a ponerle nombre a las lineas de tren
# Definimos que el nombre de la linea será CIUDAD CON MAS VIAJES DE ORIGEN - CIUDAD CON MENOS VIAJES DE ORIGEN

city_order = X_pred.origin.value_counts().index.values

def train_line(origin, destination, city_list):
  if np.where(city_order == origin) < np.where(city_order == destination):
    return origin+'-'+destination
  else:
    return destination+'-'+origin

# Aplicamos la funcion linea a linea en el dataframe
X_pred['train_line'] = X_pred.apply(lambda x: train_line(x['origin'], x['destination'], city_order), axis=1)

In [ ]:
X_pred.head()

In [ ]:
# vamos a ver las variables que más tipos tienen
100*X_pred.vehicle_class.value_counts()/X_pred.shape[0]

In [ ]:
X_pred.shape

In [ ]:
X_pred.isnull().sum()

In [ ]:
categorical = X_pred.select_dtypes(include='object').columns

In [ ]:
X_pred[categorical].nunique()

In [ ]:
for i in categorical:
  print(i)
  print(100*X_pred[i].value_counts()/X_pred.shape[0])
  print()

In [ ]:
#Definimos como residual todo valor con una frecuencia por debajo del 5%
min_weight = 0.05

for i in categorical:
  count = X_pred[i].value_counts() / X_pred.shape[0]
  most_frequent = count[count>0.05].index.to_list()
  X_pred[i] = X_pred[i].apply(lambda x: x if x in most_frequent else 'OTRO')

In [ ]:
for i in categorical:
  print(i)
  print(100*X_pred[i].value_counts()/X_pred.shape[0])
  print()

In [ ]:
# one hot coding

In [ ]:
X_pred = pd.get_dummies(data=X_pred,
                               prefix=None,
                               prefix_sep="_",
                               columns=categorical,
                               dtype="int8")

In [ ]:
X_pred.isnull().sum()

In [ ]:
X_pred.drop(columns=["departure", "arrival", "insert_date"], inplace=True)

In [ ]:
X_pred.columns

In [ ]:
X_pred.info()

In [ ]:
# variables entre si altamente correlaconados, no hay ninguna

In [ ]:
highly_corr = highly_correlated(X, y, 0.95)
highly_corr

In [ ]:
from sklearn.feature_selection import VarianceThreshold

In [ ]:
vt = VarianceThreshold(threshold = 0.01) #Eliminamos columnas donde el 99% de los valores son iguales
vt.fit(X_pred)

cols_lowvar = X_pred.columns[vt.get_support()==False]
X_pred.drop(columns=cols_lowvar,inplace=True)

print(len(cols_lowvar),' low variance features were removed:\n', cols_lowvar)

In [ ]:
# predicciones

In [ ]:
predictions = model_XGBReg.predict(X_pred)

In [ ]:
df_output2 = X_pred.copy(deep=True)
df_output2['PREDICTION Price'] = predictions
df_output2

In [ ]:
y_pred = []

In [ ]:
y_pred = df_output2['PREDICTION Price']

In [ ]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_pred, y_pred, test_size=0.2, random_state=42)

In [ ]:
# ahora estandarizamos

In [ ]:
scaler = StandardScaler()
X_train5 = scaler.fit_transform(X_train4)
X_test5 = scaler.transform(X_test4)

In [ ]:
# de nuevo usamos el método XGBReg

In [ ]:
model_XGBReg = xgb.XGBRegressor(colsample_bytree=0.5, n_estimators=500, objective='reg:squarederror')

In [ ]:
eval_set = [(X_train5, y_train4), (X_test5, y_test4)]
model_XGBReg.fit(
    X_train5,
    y_train4,
    eval_metric = "rmse",
    eval_set = eval_set,
    verbose=False
)

In [ ]:
# y finalmente obtenemos el error más pequeño de todo el dataset y por tanto subiremos el modelo final a un fichero

In [ ]:
y_pred_train = model_XGBReg.predict(X_train5)
rmse_train = np.sqrt(np.mean((y_pred_train - y_train4) ** 2))

y_pred_test = model_XGBReg.predict(X_test5)
rmse_test = np.sqrt(np.mean((y_pred_test - y_test4) ** 2))

print("RMSE Train: %.3f" %rmse_train)
print("RMSE Test: %.3f" %rmse_test)

In [ ]:
y_pred.to_csv('/content/drive/MyDrive/Notebooks proyectos/Proyecto RENFE/y_pred.csv')

**FIN**